## 문제 2-4 : 여행 계획 분석기
### 출력 파서: StructuredOutputParser

### 문제
#### 여행 후기나 계획 텍스트를 입력받아 여행지, 기간, 예산, 추천도(1-5점), 주요 활동 리스트를 구조화된 형태로 추출하는 시스템을 만드세요.

### 요구사항:
#### StructuredOutputParser와 ResponseSchema 사용
#### 5개의 필드를 정의하여 정보 추출
#### 자연어 텍스트에서 핵심 정보 파싱

### 예시 입력:
#### "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."

### 예시 출력 구조:
#### {
####    "destination": "부산",
####    "duration": "2박 3일",
####    "budget": "30만원",
####    "rating": "4",
####    "activities": ["해운대 바다구경", "자갈치시장 회 먹기", "감천문화마을 구경"]
#### }


In [3]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:3])

gsk


In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

In [27]:
# 출력 구조 정의 (지역, 기간, 예산, 추천도, 주요 활동)
response_schemas = [
    ResponseSchema(name="destination", description="여행지(도시 또는 지역명, 예: 부산)"),
    ResponseSchema(name="duration",    description="여행 기간(자연어, 예: 2박 3일)"),
    ResponseSchema(name="budget",      description="예산(자연어, 예: 30만원/50만 원 등 통화 포함 가능)"),
    ResponseSchema(name="rating",      description="추천도(1~5 사이 숫자, 정수 한 개)"),
    ResponseSchema(name="activities",  description="주요 활동 리스트(간결한 한국어 명사/구, 예: ['해운대 바다구경','회 먹기'])"),
]

In [6]:
# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print("출력 형식 지시사항:")
print(format_instructions)

출력 형식 지시사항:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"destination": string  // 여행지(도시 또는 지역명, 예: 부산)
	"duration": string  // 여행 기간(자연어, 예: 2박 3일)
	"budget": string  // 예산(자연어, 예: 30만원/50만 원 등 통화 포함 가능)
	"rating": string  // 추천도(1~5 사이 숫자, 정수 한 개)
	"activities": string  // 주요 활동 리스트(간결한 한국어 명사/구, 예: ['해운대 바다구경','회 먹기'])
}
```


In [14]:
# 프롬프트 템플릿
template = """
    너는 정보 추출기야. 사용자가 제공한 여행 후기,계획 텍스트에서 핵심 정보를 구조화하여 추출해.\n
    반드시 아래 출력 형식을 그대로 따르고, 값이 애매하면 문맥상 가장 그럴듯한 값을 한 개로 정제해.\n
	
    리뷰 내용: {review}
    
    {format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

In [15]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="openai/gpt-oss-120b",
    temperature=0.1,
)

In [16]:
review = """
	지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데
    해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요.
    정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요.
"""

In [17]:
# 체인 실행
chain = prompt | llm | parser

output = chain.invoke({"review": review})

In [28]:
print("===== 분석 결과 =====")
print(f"지역: {output['destination']}")
print(f"기간: {output['duration']}")
print(f"예산: {output['budget']}")
print(f"추천도: {output['rating']}")
print(f"주요 활동: {output['activities']}")

===== 분석 결과 =====
지역: 부산
기간: 2박 3일
예산: 30만원
추천도: 4
주요 활동: ['해운대 바다구경','자갈치시장 회 먹기','감천문화마을 구경']
